In [21]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from loguru import logger

from src.text_embedder.embedder import EmbeddingsModel

from pathlib import Path
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [22]:
notebook_path = Path().resolve().parent
print(notebook_path)

# Construct the filename relative to the new path
filename = notebook_path / "test_frames_math1.csv"

# Read the CSV file
df = pd.read_csv(filename)
print(df)

C:\Users\baatout\PycharmProjects\afm-vlm\experiments\second_experiment
    ID                                             Prompt Length  GT_Keyframe
0    1       what is a binomial probability distribution?   long           18
1    2    x is a specific number of successes in n trials   long           25
2    3                                       P(F) = 1 -p   short           23
3    4  what procedures does the binomial probability ...   long           29
4    5          What is the binomial probability formula?   long           58
5    6  What is the method 2 for finding the binomial ...   long           80
6    7  How many NFL Football games betweem 1974 and 2...   long           85
7    8           What are the key concepts of the lecture   long            3
8    9  What is the probability of failure in the twit...   long           46
9   10                Using the given values of n x and q   long           65
10  11  probability of success remains the same in all...   long       

In [23]:
# create dataframe to save results of the experiments
columns = ['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3']

df_test = pd.DataFrame(columns=columns)

df_ocr_only = df_test
df_ocr_lava = df_test
df_ocr_transcriptions = df_test

df_short_llm_summary = df_test
df_extensive_summary = df_test

In [24]:
def add_to_df(prompt, gt_keyframe, result):
    # Create a new row with the provided data
    return {
        'Prompt': prompt,
        'GT_Keyframe': gt_keyframe,
        'Top_1': extract_keyframe_number(result[0]) if len(result) > 0 else None,
        'Top_2': extract_keyframe_number(result[1]) if len(result) > 1 else None,
        'Top_3': extract_keyframe_number(result[2]) if len(result) > 2 else None
    }


# ietrate over the dataframe and get the results
def get_results(df):
    for _, row in df.iterrows():
        logger.info(row['Prompt'])
        prompt = row['Prompt']
        gt_keyframe = row['GT_Keyframe']

        # Search for similar images
        output = embedding_model.search_similar_images_top_k(prompt, gt_keyframe, 3)
        res_row = add_to_df(prompt, gt_keyframe, output)
        rows.append(res_row)

    return rows


def extract_keyframe_number(path):
    """
    Extracts the scene number from the given file path.

    Parameters:
    path (str): The full path of the file.

    Returns:
    str: The extracted scene number.
    """
    # Get filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]

    # Extract '032' from filename
    scene_number = filename.split('-Scene-')[-1].split('-')[0]

    return scene_number

In [25]:
#top_3 = []
#embedding_model.check_proximity_keyframes(90, top_3)

# LOAD MODEL 

In [26]:
embedding_model = EmbeddingsModel()

<Figure size 800x2000 with 0 Axes>

In [27]:
csv = notebook_path / "math_1.csv"
df_data_extensive = pd.read_csv(csv)
df_data_extensive

,Unnamed: 0,80,26,38,45,34,49,57,2,61,...,40,23,76,15,68,19,64,7,89,85
0,img_path,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...
1,timestamps,"[1454.08, 1459.2]","[399.44, 404.48]","[715.6, 716.92]","[783.36, 785.6]","[593.92, 599.04]","[798.72, 813.96]","[977.92, 1083.88]","[10.12, 30.72]","[1190.28, 1231.16]",...,"[721.92, 763.32]","[313.64, 369.36]","[1408.0, 1433.6]","[117.76, 150.28]","[1345.12, 1346.56]","[209.92, 252.48]","[1264.64, 1306.4]","[61.44, 63.04]","[1820.88, 1861.8]","[1617.92, 1679.88]"
2,transcription,Another way to find binomial probabilities is...,Notice that X can be any whole number between...,I guess they'd set with RIP,"equals 5, x equals 3.","So here that is again, again the example we'r...",that X and P both refer to the same concept o...,"Right, I would be very surprised if anybody i...","Okay, so the binomial distribution, binomial ...","Okay, so back to the Twitter example, and the...",...,C major is a binomial distribution. So let's...,"Okay. Notation. So, P little P is what we alw...",used your specific number or whatever you got...,answers right or wrong. Number three is D. Nu...,NaN,"It has four requirements. Number one, the pr...","Okay, now I'm just going to go ahead and hit ...",NaN,Our sort of working definition of what is an ...,"Okay, so here's a good example for technology..."
3,ocr_extracted_text,Method 2: Using Technology\r\n\r\nTechnologies...,"be any whole number between 0 and n, inclusive...",The word success as used here is arbitrary and...,Solution\r\n\r\nb. Having concluded that the g...,* Binomial Probability Distribution BS\r\n—Abi...,"ple:\r\n\r\nSolution\r\n\r\nAgain, it is very ...",Let’s develop the fo\r\n\r\ni\r\n\r\n+ On the ...,Key Concept\r\n\r\nThe focus of this section i...,Example: Twitter (7 or\r\n\r\nGiven that there...,...,®\r\n\r\nple:\r\n\r\nWhen an adult is randomly...,Sand F (success and failure) denote the two\r\...,X=number Frequency Relative freq. Theoretical ...,The Answers — how did you do?\r\n\r\n,2\r\n\r\n+ How many did you get right on the p...,ab\r\n\r\n* Binomial Probability Distribution\...,NaN,Pop quiz! i\r\n\r\n* Number your paper 1-10. T...,Solution\r\n\r\nThe Excel display on the next ...,We previously noted that between 1974 and 2011...
4,llava_result,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The slide appears to be from an academic lect...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The image you've provided appears to be a scr...,The image you've provided app

In [28]:
# get llm_long_summary 
llm_long_summary = df_data_extensive.iloc[6].to_dict().values()

# convert to list
llm_long_summary = list(llm_long_summary)
llm_long_summary = llm_long_summary[1:]
logger.info(llm_long_summary)

2024-07-13 13:08:53.350 | INFO     | __main__:<module>:7 - ['Method 2: Using Technology - The lecture discussed how technology can be used to find binomial probabilities. It mentioned that technologies such as spreadsheets and calculators can help in this process. The slide provided an example of a binomial probability, where there are two possible outcomes (success or failure) and the probability of each outcome is known. The slide also included a visual aid, which seems to be a screenshot of a spreadsheet with a formula for calculating the probability of a binomial event.', 'The slide explains notation for binomial probability distributions, which model the number of successes in a series of independent trials. The notation includes P(x) - probability of getting exactly x successes among n trials, where 0 ≤ x ≤ n. The probability of success (P) and failure (q) are also defined. The bionomial formula is given as P(x; n) = (n choose x) * p^x * (1 - p)^(n - x), where p is the probabilit

In [29]:
# get ocr_only
# Since ocr_only embeddings can't be mapped to floats due to e-02, e-01, we're generating new embeddings 
ocr_only = df_data_extensive.iloc[3].to_dict().values()

# convert to list
ocr_only = list(ocr_only)

ocr_only = ocr_only[1:]

for i in ocr_only:
    logger.info(i)

2024-07-13 13:08:53.626 | INFO     | __main__:<module>:11 - Method 2: Using Technology

Technologies can be used to find binomial probabilities. The
screen displays on the next slide list binomial probabilities for
n=5 and p = 0.85, as in the previous example. Notice that in
each display, the probability distribution is given as a table.

Copyright © 2038, 2014, 2012 Pearton Education,


2024-07-13 13:08:53.633 | INFO     | __main__:<module>:11 - be any whole number between 0 and n, inclusive
Pp - probability of success in one of the n trials

q- probability of failure in one of the n trials

P(x) - probability of getting exactly x successes among

the n trials

CCopyiht © 2048, 2014, 2012 Pearson Education, Ine. Al Rights Reserved

2024-07-13 13:08:53.633 | INFO     | __main__:<module>:11 - The word success as used here is arbitrary and does
not necessarily represent something good. Either of the
two possible categories may be called the success S as
long as its probability is identif

In [30]:
# get ocr_lava 
lava_only = df_data_extensive.iloc[4].to_dict().values()

lava_only = list(lava_only)

lava_only = lava_only[1:]

concat_result_ocr_lava = []

# list to a list of strings
ocr_only = [str(i) for i in ocr_only]

lava_only = [str(i) for i in lava_only]

for i in range(len(ocr_only)):
    concat_result_ocr_lava.append(ocr_only[i] + lava_only[i])

In [31]:
# get ocr_transcriptions
transcriptions = df_data_extensive.iloc[2].to_dict().values()

transcriptions = list(transcriptions)
transcriptions = transcriptions[1:]

concat_result_ocr_transcriptions = []

# concatenate the result 
ocr = ocr_only

transcriptions = [str(i) for i in transcriptions]
ocr_only = [str(i) for i in ocr_only]

# for i in transcriptions:
#     if transcriptions[i] == 'nan' or transcriptions[i] == None:
#         transcriptions[i] = ''

for i in range(len(ocr_only)):
    concat_result_ocr_transcriptions.append(ocr_only[i] + transcriptions[i])

logger.info(concat_result_ocr_transcriptions)

2024-07-13 13:08:54.549 | INFO     | __main__:<module>:22 - ['Method 2: Using Technology\r\n\r\nTechnologies can be used to find binomial probabilities. The\r\nscreen displays on the next slide list binomial probabilities for\r\nn=5 and p = 0.85, as in the previous example. Notice that in\r\neach display, the probability distribution is given as a table.\r\n\r\nCopyright © 2038, 2014, 2012 Pearton Education,\r\n\r\n Another way to find binomial probabilities is using technology.', 'be any whole number between 0 and n, inclusive\r\nPp - probability of success in one of the n trials\r\n\r\nq- probability of failure in one of the n trials\r\n\r\nP(x) - probability of getting exactly x successes among\r\n\r\nthe n trials\r\n\r\nCCopyiht © 2048, 2014, 2012 Pearson Education, Ine. Al Rights Reserved\r\n Notice that X can be any whole number between zero.', "The word success as used here is arbitrary and does\r\nnot necessarily represent something good. Either of the\r\ntwo possible categorie

In [32]:
# get img_path
img_path = df_data_extensive.iloc[0].to_dict().values()

# convert to list
img_path = list(img_path)
img_path = img_path[1:]

In [33]:
# Embedd with LLM long summary
logger.info(f"Embedd with LLM long summary")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(llm_long_summary)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_extensive_summary = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_extensive_summary.to_csv('df_standard_llm_long_summary_math.csv', index=False)


2024-07-13 13:08:55.362 | INFO     | __main__:<module>:2 - Embedd with LLM long summary
2024-07-13 13:09:02.844 | INFO     | __main__:<module>:18 - Embedding Model.text_embeddings: tensor([[-0.1807,  0.0187, -0.0795,  ..., -0.0288, -0.0470,  0.1070],
        [-0.0751, -0.1122, -0.1110,  ...,  0.1050,  0.0080, -0.0782],
        [ 0.1494, -0.0069, -0.1828,  ...,  0.1257, -0.1049,  0.2026],
        ...,
        [ 0.0975,  0.2250,  0.0211,  ...,  0.3876, -0.1169,  0.2771],
        [ 0.1336, -0.2133, -0.2561,  ...,  0.2625, -0.0250, -0.1052],
        [ 0.0451, -0.1206, -0.2157,  ...,  0.3979, -0.0515,  0.0212]])
2024-07-13 13:09:02.859 | INFO     | __main__:get_results:15 - what is a binomial probability distribution?
2024-07-13 13:09:02.890 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:157 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.8725, 0.7878, 0.7841]),
indices=tensor([ 7,  4, 69])) - GT is keyframe number 18
2024-07-13 13:09:02.890 | INFO 

<Figure size 800x2000 with 0 Axes>

In [34]:
# Embedd with OCR ONLY 
logger.info(f"Embedd with OCR ONLY ")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

# need to be embedded

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(ocr_only)

embedding_model.text_embeddings = embeddings

# already embedded 
#
# if isinstance(embedding_model.text_embeddings, list):
#     for i, text_embedding in enumerate(embedding_model.text_embeddings):
#         logger.info(text_embedding)
#         # Regular expression pattern to find numeric values
#         numeric_pattern = r"[-+]?\d*\.\d+(?:[eE][-+]?\d+)?"
#         
#         # Extract numeric values from the tensor string
#         tensor_values_str = re.findall(numeric_pattern, text_embedding)
#         
#         # Convert the extracted string values to float
#         tensor_values = [float(value) for value in tensor_values_str]
#         
#         # Create a PyTorch tensor from the list of floats
#         embedding_model.text_embeddings[i] = torch.tensor(tensor_values)
#         
# 
# # create one single torch for sim search 
# embedding_model.text_embeddings = torch.stack(embedding_model.text_embeddings, dim=0)
# 
# logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_ocr_only = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_only.to_csv('df_standard_ocr_only_math.csv', index=False)
logger.info('Saved old_df_standard_ocr_only_math.csv')

2024-07-13 13:09:03.817 | INFO     | __main__:<module>:2 - Embedd with OCR ONLY 
2024-07-13 13:09:11.194 | INFO     | __main__:get_results:15 - what is a binomial probability distribution?
2024-07-13 13:09:11.352 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:157 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.7391, 0.7355, 0.7067]),
indices=tensor([54,  0, 31])) - GT is keyframe number 18
2024-07-13 13:09:11.357 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:159 - Length of img paths: 89
2024-07-13 13:09:11.357 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - #####GT is keyframe number 18#####
2024-07-13 13:09:11.357 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Max similarity for index 54 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/math_1/extracted_keyframes/math_1-Scene-081-01.jpg
2024-07-13 13:09:11.361 | INFO     | src.text_embedder.embedder:se

<Figure size 800x2000 with 0 Axes>

In [35]:
# Embedd with OCR TRANSCRIPTIONS
# TODO: Doesn't work since transcriptions are nans
logger.info(f"Embedd with TRANSCRIPTIONS")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(concat_result_ocr_transcriptions)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_ocr_transcriptions = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_transcriptions.to_csv('df_standard_df_ocr_transcriptions_math.csv', index=False)

2024-07-13 13:09:13.042 | INFO     | __main__:<module>:3 - Embedd with TRANSCRIPTIONS
2024-07-13 13:09:23.252 | INFO     | __main__:<module>:19 - Embedding Model.text_embeddings: tensor([[-0.1558, -0.0281, -0.0629,  ..., -0.0501, -0.1999,  0.0217],
        [-0.0288, -0.0639, -0.1174,  ...,  0.2123,  0.0999, -0.0971],
        [ 0.0047, -0.1774, -0.1902,  ...,  0.1141,  0.0542,  0.1336],
        ...,
        [-0.1746,  0.2062,  0.1619,  ...,  0.2807, -0.0483,  0.2825],
        [ 0.1472, -0.0332, -0.2890,  ...,  0.1943,  0.0194, -0.0999],
        [ 0.0034, -0.1747, -0.1204,  ...,  0.2931,  0.0011, -0.0896]])
2024-07-13 13:09:23.252 | INFO     | __main__:get_results:15 - what is a binomial probability distribution?
2024-07-13 13:09:23.348 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:157 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.7256, 0.7189, 0.7067]),
indices=tensor([ 0, 54, 31])) - GT is keyframe number 18
2024-07-13 13:09:23.348 | INFO   

<Figure size 800x2000 with 0 Axes>

In [36]:
# Embedd with OCR LAVA
logger.info(f"Embedd with OCR LAVA")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(concat_result_ocr_lava)

embedding_model.text_embeddings = embeddings

rows = []

rows = get_results(df)

df_ocr_lava = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_lava.to_csv('df_standard_ocr_lava_math.csv', index=False)

2024-07-13 13:09:25.415 | INFO     | __main__:<module>:2 - Embedd with OCR LAVA
2024-07-13 13:09:36.505 | INFO     | __main__:get_results:15 - what is a binomial probability distribution?
2024-07-13 13:09:36.584 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:157 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.7246, 0.7208, 0.7160]),
indices=tensor([28, 42, 54])) - GT is keyframe number 18
2024-07-13 13:09:36.584 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:159 - Length of img paths: 89
2024-07-13 13:09:36.584 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - #####GT is keyframe number 18#####
2024-07-13 13:09:36.584 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Max similarity for index 28 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/math_1/extracted_keyframes/math_1-Scene-020-01.jpg
2024-07-13 13:09:36.584 | INFO     | src.text_embedder.embedder:sea

<Figure size 800x2000 with 0 Axes>